## Emotion recognition using NLP: Affective Computing 2023

Danila Goncharenko, 2303788

Ana Ferreira, 2308587

Luca Hustiuc, 2209104

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("tweet_emotions.csv")

In [3]:
from transformers import AutoModelForSequenceClassification, pipeline, AutoTokenizer

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

model_nm = "cardiffnlp/twitter-roberta-base"
fill_mask = pipeline("fill-mask", model=model_nm, tokenizer=model_nm)
tokenizer = AutoTokenizer.from_pretrained(model_nm)

def print_candidates():
    for i in range(5):
        token = tokenizer.decode(candidates[i]['token'])
        score = np.round(candidates[i]['score'], 4)
        print(f"{i+1}) {token} {score}")

texts = [
 "I am so <mask> 😊",
 "I am so <mask> 😢" 
]

for text in texts:
    t = preprocess(text)
    print(f"{'-'*30}\n{t}")
    candidates = fill_mask(t)
    print_candidates()


c:\Users\utilizador.DESKTOP-DGA4C5V.000\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 565/565 [00:00<?, ?B/s] 
Downloading:   9%|▉         | 46.2M/501M [00:07<00:38, 11.7MB/s]

KeyboardInterrupt: 

Downloading:   9%|▉         | 46.6M/501M [00:18<00:38, 11.7MB/s]

In [ ]:
model_nm = 'ai-forever/ruBert-base'

from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import numpy as np

MODEL = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

text = "Good night 😊"
text = preprocess(text)

# Pytorch
model = AutoModel.from_pretrained(MODEL)
encoded_input = tokenizer(text, return_tensors='pt')
features = model(**encoded_input)
features = features[0].detach().cpu().numpy() 
features_mean = np.mean(features[0], axis=0) 
#features_max = np.max(features[0], axis=0)

# # Tensorflow
# model = TFAutoModel.from_pretrained(MODEL)
# encoded_input = tokenizer(text, return_tensors='tf')
# features = model(encoded_input)
# features = features[0].numpy()
# features_mean = np.mean(features[0], axis=0) 
# #features_max = np.max(features[0], axis=0)


In [1]:
from datasets import load_dataset
dataset = load_dataset("Djacon/ru_goemotions")

def tok_func(x): return tokz(x['text'], truncation=True)

LABELS = ['joy', 'interest', 'surpise', 'sadness', 'anger', 'disgust', 'fear', 'guilt', 'neutral']

def binarize_labels(labels):
    return [int(i in list(map(int,labels[1:-1].split(',')))) for i in range(len(LABELS))]
binarize_labels(dataset['train']['labels'][0])




C:\Users\Dan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2710.08it/s]
Generating train split: 26945 examples [00:00, 157760.06 examples/s]
Generating validation split: 3294 examples [00:00, 131377.36 examples/s]
Generating test split: 3367 examples [00:00, 136136.17 examples/s]


[0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
# train
# validate
# test

model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=len(LABELS), problem_type="multi_label_classification")

model.config.label2id = {label: i for i, label in enumerate(LABELS)}
model.config.id2label = {i: label for i, label in enumerate(LABELS)}

train_ds = dataset['train'].map(tok_func, batched=True).map(lambda x: {'label': [float(y) for y in binarize_labels(x['labels'])]}, batched=False, remove_columns=['text', 'labels'])
val_ds = dataset['validation'].map(tok_func, batched=True).map(lambda x: {'label': [float(y) for y in binarize_labels(x['labels'])]}, batched=False, remove_columns=['text', 'labels'])
test_ds = dataset['test'].map(lambda x: {'label': [float(y) for y in binarize_labels(x['labels'])]}, batched=False, remove_columns=['labels'])

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from torch import nn
def get_pred(text):
    inputs = tokz(text, return_tensors="pt")
    inputs.to(model.device)
    with torch.no_grad():
        logits = model(**inputs).logits
    percent = nn.functional.softmax(logits, dim=1)
    predicted_class_id = logits.argmax().item()
    res = model.config.id2label[predicted_class_id]
    return percent, predicted_class_id, res
get_pred('This text has no emotion')

In [ ]:
from tqdm.auto import tqdm, trange
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support
def predict_with_model(model, dataloader):
    preds = []
    facts = []

    for batch in tqdm(dataloader):
        facts.append([list(map(bool,batch['label']))])
        inputs = tokz(batch['text'], return_tensors="pt")
        inputs.to(model.device)
        with torch.no_grad():
            logits = model(**inputs).logits
        preds.append(nn.functional.softmax(logits, dim=1).cpu())
    facts = np.concatenate(facts)
    preds = np.concatenate(preds)
    return facts, preds
def eval_model(preds, facts):
    aucs = [roc_auc_score(facts[:, i], preds[:, i]) for i in range(len(LABELS))]
    print('aucs:', aucs)
    return {'accuracy': np.mean(aucs)}

In [ ]:
factswm, predwm = predict_with_model(model, test_ds.select(range(100)))
eval_model(predwm, factswm)

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
def compute_metrics(eval_pred):
    preds, facts = eval_pred
    return eval_model(preds, facts)

In [ ]:
training_args = TrainingArguments(
    output_dir="test",
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    num_train_epochs=5, 
    weight_decay=0.01,
    evaluation_strategy="epoch", 
    save_strategy="epoch", 
    metric_for_best_model = "accuracy", 
    load_best_model_at_end=True, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokz,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
get_pred('This text has no emotion.')

In [ ]:
get_pred('Im afraid Ill fail')

In [ ]:
get_pred('Glad to see you!')

In [ ]:
get_pred('I want to die')

In [ ]:
get_pred('OMG, so romantic')

In [ ]:
from sklearn.metrics import f1_score
pd.DataFrame([
    {av: f1_score(factswm[:, i], predwm[:, i] > 0.5, average=av) for av in ['binary', 'micro', 'macro']}
    for i in range(len(LABELS))
]).round(4)

In [ ]:
pd.DataFrame([
    {av: f1_score(factswm[:, i], predwm[:, i] > 0.5, average=av) for av in ['binary', 'micro', 'macro']}
    for i in range(len(LABELS))
]).mean().round(4)